In [7]:
import pandas as pd
import re
import nltk
import os
from pathlib import Path



unc_remove_re = re.compile(r'\W+')
corp_re = re.compile('( (group|holding(s)?( co)?|inc(orporated)?|ltd|l ?l? ?[cp]|co(rp(oration)?|mpany)?|s[ae]|plc))+$')
and_re = re.compile(' & ')
punc1_re = re.compile(r'(?<=\S)[\'’´\.](?=\S)')
punc2_re = re.compile(r'[\s\.,:;/\'"`´‘’“”\(\)\[\]\{\}_—\-?$=!]+')

STOPWORDS = nltk.corpus.stopwords.words('english')
STOPWORDS.remove("am")
STOPWORDS.remove("up")
STOPWORDS.remove("in")
STOPWORDS.remove("on")
STOPWORDS.remove("all")
STOPWORDS.remove("any")
STOPWORDS.remove("most")
STOPWORDS.remove("no")
STOPWORDS.remove("nor")
STOPWORDS.remove("own")
STOPWORDS.remove("same")
STOPWORDS.remove("so")
STOPWORDS.remove("very")
STOPWORDS.remove("s")
STOPWORDS.remove("t")
STOPWORDS.remove("d")
STOPWORDS.remove("ll")
STOPWORDS.remove("m")
STOPWORDS.remove("o")
STOPWORDS.remove("re")
STOPWORDS.remove("ve")
STOPWORDS.remove("y")

stopword_re_str = r""
for word in STOPWORDS:
	stopword_re_str += r'\b' + word + r'\b|'
stopword_re = re.compile(stopword_re_str[:-1]) # The negative 1 is for the fencepost |


# Function to calculate longest common substring, from https://www.geeksforgeeks.org/print-longest-common-substring/
# function to find and print 
# the longest common substring of
# X[0..m-1] and Y[0..n-1]
def get_longest_common_substring(X, Y, m, n):
 
    # Create a table to store lengths of
    # longest common suffixes of substrings.
    # Note that LCSuff[i][j] contains length
    # of longest common suffix of X[0..i-1] and
    # Y[0..j-1]. The first row and first
    # column entries have no logical meaning,
    # they are used only for simplicity of program
    LCSuff = [[0 for i in range(n + 1)]
                 for j in range(m + 1)]
 
    # To store length of the
    # longest common substring
    length = 0
 
    # To store the index of the cell
    # which contains the maximum value.
    # This cell's index helps in building
    # up the longest common substring
    # from right to left.
    row, col = 0, 0
 
    # Following steps build LCSuff[m+1][n+1]
    # in bottom up fashion.
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                LCSuff[i][j] = LCSuff[i - 1][j - 1] + 1
                if length < LCSuff[i][j]:
                    length = LCSuff[i][j]
                    row = i
                    col = j
            else:
                LCSuff[i][j] = 0
 
    # if true, then no common substring exists
    if length == 0:
        return ""
 
    # allocate space for the longest
    # common substring
    resultStr = ['0'] * length
 
    # traverse up diagonally form the
    # (row, col) cell until LCSuff[row][col] != 0
    while LCSuff[row][col] != 0:
        length -= 1
        resultStr[length] = X[row - 1] # or Y[col-1]
 
        # move diagonally up to previous cell
        row -= 1
        col -= 1
 
    # required longest common substring
    longest_common_substring = ''.join(resultStr)

    return longest_common_substring


# Function from Brad Hackinen's NAMA
def basicHash(s):
    '''
    A simple case and puctuation-insensitive hash
    '''
    s = s.lower()
    s = re.sub(and_re,' and ',s)
    s = re.sub(punc1_re,'',s)
    s = re.sub(punc2_re,' ',s)
    s = s.strip()

    return s

# Function from Brad Hackinen's NAMA
def corpHash(s):
    '''
    A hash function for corporate subsidiaries
    Insensitive to
        -case & punctation
        -'the' prefix
        -common corporation suffixes, including 'holding co'
    '''
    s = basicHash(s)
    if s.startswith('the '):
        s = s[4:]

    s = re.sub(corp_re,'',s,count=1)

    return s

# function to clean org names
def clean_fin_org_names(name):
    if name is None or not isinstance(name, str) or name == "NA":
        return ""
    else:
        # James strip metadata from name
        name = name.split(',')[0]
        name = re.sub(" [0-9]* [k|m]b pdf","",name)

        # name = name.translate(corp_simplify_utils.STR_TABLE)
        # comment out this line for now since corp_simplify_utils is not imported
        # TO DO: import corp_simplify_utils if needed, and ask for access if it's private
        name = re.sub(stopword_re, '', name.lower())
        
        return corpHash(name)

In [ ]:
# Robustly locate the data directory by checking likely locations
expected_files = [
	'CompustatNames.csv',
	'CIK.csv',
	'FDIC_clean.csv',
	'SEC_Institutions.csv',
]

candidates = [
	Path.cwd() / 'data',               # e.g., .../corporations-data/data
	Path.cwd().parent / 'data',        # e.g., .../urop-project/data
	Path.cwd().parent.parent / 'data', # one more level up if needed
]

data_dir = None
for cand in candidates:
	if cand.exists() and any((cand / f).exists() for f in expected_files):
		data_dir = cand.resolve()
		break

if data_dir is None:
	raise FileNotFoundError(
		f"None of the expected CSV files were found. Checked: {candidates}"
	)

compustat_df = pd.read_csv(data_dir / 'CompustatNames.csv')
cik_df = pd.read_csv(data_dir / 'CIK.csv')
fdic_df = pd.read_csv(data_dir / 'FDIC_clean.csv')
sec_df = pd.read_csv(data_dir / 'SEC_Institutions.csv')

# cleaning and standardizing organization names
compustat_df['std_name'] = compustat_df['conm'].apply(clean_fin_org_names)
fdic_df['std_name'] = fdic_df['NAME'].apply(clean_fin_org_names)
sec_df['std_name'] = sec_df['Name'].apply(clean_fin_org_names)
cik_df['std_name'] = cik_df['company_name'].apply(clean_fin_org_names)

print(cik_df[['company_name', 'std_name']].head(10))
print(sec_df[['Name', 'std_name']].head(10))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/stevenkang/Documents/urop-project/data/CompustatNames.csv'